In [ ]:
# Import the GLMakie library
# This provides all the functions for plotting and window creation.
using GLMakie

# --- 1. Set up the Scene and Theme ---

# Use a dark theme, similar to the HTML example
set_theme!(theme_dark())

# Create a Figure (the main window) and an Axis (the plotting area)
fig = Figure(resolution = (600, 600))
ax = Axis(
    fig[1, 1],
    # Set the background color explicitly
    backgroundcolor = :black,
    # Set the aspect ratio to 1 to make it a perfect square
    aspect = DataAspect(),
    # Hide the default grid lines
    xgridvisible = false,
    ygridvisible = false
)

# Set fixed limits for the axes so the view doesn't jump around
limits!(ax, -300, 300, -300, 300)

# Hide the default axis decorations (numbers, ticks, labels)
hidedecorations!(ax)

# Draw the X and Y axes manually
lines!(ax, [-300, 300], [0, 0], color = :gray, linewidth = 1) # X-Axis
lines!(ax, [0, 0], [-300, 300], color = :gray, linewidth = 1) # Y-Axis


# --- 2. Set up Observables for Animation ---

# "Observables" are special variables from Makie.
# When you change an Observable's value, any plot
# connected to it will automatically update.

const vector_length = 250.0
const rotation_speed = 0.01

# Create an Observable for the angle. We start it at 0.0.
angle = Observable(0.0)

# Create Observables for the vector's (x, y) endpoint.
# We use `lift` to tie them to the `angle`.
# Whenever `angle` changes, `end_x` and `end_y` will be
# automatically recalculated.
end_x = lift(a -> vector_length * cos(a), angle)
end_y = lift(a -> vector_length * sin(a), angle)

# `arrows!` is a Makie function to draw arrows.
# It takes an origin point and a "direction" vector.
# We use `lift` again to create the direction vector Observable.
vector_direction = lift((x, y) -> [Point2f(x, y)], end_x, end_y)
vector_origin = Observable([Point2f(0, 0)])

# --- 3. Plot the Vector and Text ---

# Plot the main vector as an arrow.
# Because `vector_direction` is an Observable,
# this arrow will update automatically.
arrows!(
    ax,
    vector_origin,      # Start at (0, 0)
    vector_direction,   # Point to (x, y)
    color = :emerald,   # Similar to the green in the other example
    linewidth = 4,
    arrowsize = 20
)

# Create an Observable for the info text
info_text = lift(end_x, end_y, angle) do x, y, a
    angle_deg = (a * 180 / pi) % 360
    """
    x: $(round(x, digits=2))
    y: $(round(y, digits=2))
    Angle (θ): $(round(angle_deg, digits=1))°
    """
end

# Plot the text. We position it in the top-left corner.
text!(
    ax,
    info_text,
    position = Point2f(-280, 280),
    align = (:left, :top),
    color = :white,
    fontsize = 16
)

# --- 4. The Animation Loop ---

println("Starting animation... Close the window to stop.")

# `fig.scene.events.window_open` is an Observable
# that is `true` while the window is open and `false` when
# you close it.
while fig.scene.events.window_open[]
    # This is the core of the animation!
    # By changing the value of the `angle` Observable,
    # all the plots that `lift` from it will auto-update.
    angle[] = angle[] + rotation_speed

    # sleep(0.016) is approx 60 FPS.
    # `yield()` is often better as it lets Makie
    # process events and render smoothly.
    yield()
end

println("Animation window closed.")